In [11]:
import pandas as pd
import numpy as np
from scipy import stats
from pyts.image import GramianAngularField, RecurrencePlot, MarkovTransitionField
from sklearn.metrics.pairwise import pairwise_distances
from PIL import Image
import os

In [45]:
def preprocess_dataframe(df):
    df = df.iloc[:, 1:]
    df.columns = ['time', 'signal_val', 'label']
    df = df.loc[df.time.notnull()].copy()
    df.time = (df.time * 1000).astype('int')
    df.label = df.label.astype('int')
    return df

def create_dirs_for_subject(subject, window_intervals,
                            types= ['gaf', 'rp', 'mtf'],
                            classes = ['0', '1', '2', '3', '4']):
    for w in window_intervals:
        for c in classes:
            for t in types:
                directory = os.path.join('rml', subject, str(w) + '_seconds', c, t)
                if not os.path.exists(directory):
                    os.makedirs(directory)

In [58]:
def recurrence_plot(s, eps=None, steps=None):
    result = []
    if eps==None: eps=0.1
    if steps==None: steps=10
    d = pairwise_distances(s[:, None])           
    d = d / eps
    d[d > steps] = steps
    return d/5. - 1

def markov_transition_field(mtf, x):
    x = mtf.transform(x)
    return 2 * x - 1

def get_image(x):
    x = ((x + 1.0) * 127.5).astype('uint8')
    image = Image.fromarray(x)
    
    if image.size[0] < 224:
        image = image.resize((224, 224), resample=Image.BILINEAR)

    return image

def save_image(x, dir_prefix, subject, window_interval, image_type, image_no, label):
    x = get_image(x)
    x.save(os.path.join(dir_prefix, subject,
                            str(window_interval) + '_seconds', 
                            str(label), image_type, str(image_no)) + '.png')

In [59]:
def generate_image(subject, min_time, max_time, window_intervals, min_ts_lens, root_dir):
    for window_interval in window_intervals:
        w = int(window_interval * 1000)
        gasf = GramianAngularField(image_size=min_ts_lens[window_interval], method='difference')
        mtf = MarkovTransitionField(image_size=min_ts_lens[window_interval])
        
        c = 1
        for i in range(min_time, max_time + 1, w):
            start = i
            end = i + w
            ts = df[(df.time >= start) & (df.time < end)].signal_val.values

            if len(ts) >= min_ts_lens[window_interval]:
                labels = df[(df.time >= start) & (df.time < end)].label.values
                label = stats.mode(labels)[0][0]
                
                gaf = gasf.transform(np.expand_dims(ts, axis=0)).squeeze(axis=0)

                rp = recurrence_plot(ts, steps=10)
                rp = Image.fromarray(rp)
                rp = rp.resize((min_ts_lens[window_interval], min_ts_lens[window_interval]),
                               resample=Image.BILINEAR) 
                rp = np.asarray(rp)

                mt = markov_transition_field(mtf, np.expand_dims(ts, axis=0)).squeeze(axis=0)

                save_image(gaf, root_dir, subject, window_interval, 'gaf', c, label)
                save_image(rp, root_dir, subject, window_interval, 'rp', c, label)
                save_image(mt, root_dir, subject, window_interval, 'mtf', c, label)
                c += 1

In [60]:
for filename in os.listdir('DenoisedNormalizedData'):
    if filename.endswith(".csv"):
        subject = filename.split('_')[0]
        df = pd.read_csv('DenoisedNormalizedData/' + filename, header=None)
        df = preprocess_dataframe(df)
        window_intervals = [0.25, 0.5, 1, 2, 3, 5, 10, 15, 20, 25, 30]
        min_ts_lens = {
            0.25: 50,
            0.5: 100,
            1: 224,
            2: 224,
            3: 224,
            5: 224,
            10: 224,
            15: 224,
            20: 224,
            25: 224,
            30: 224
        }
        create_dirs_for_subject(subject, window_intervals)
        generate_image(subject, df.time.min(), df.time.max(), window_intervals, min_ts_lens, 'rml')

/home/asif/anaconda3/envs/pytorch/lib/python3.7/site-packages/pyts/preprocessing/discretizer.py:161: UserWarning: Some quantiles are equal. The number of bins will be smaller for sample [0]. Consider decreasing the number of bins or removing these samples.
  "of bins or removing these samples.".format(samples))
/home/asif/anaconda3/envs/pytorch/lib/python3.7/site-packages/pyts/preprocessing/discretizer.py:161: UserWarning: Some quantiles are equal. The number of bins will be smaller for sample [0]. Consider decreasing the number of bins or removing these samples.
  "of bins or removing these samples.".format(samples))
/home/asif/anaconda3/envs/pytorch/lib/python3.7/site-packages/pyts/preprocessing/discretizer.py:161: UserWarning: Some quantiles are equal. The number of bins will be smaller for sample [0]. Consider decreasing the number of bins or removing these samples.
  "of bins or removing these samples.".format(samples))
/home/asif/anaconda3/envs/pytorch/lib/python3.7/site-packages

/home/asif/anaconda3/envs/pytorch/lib/python3.7/site-packages/pyts/preprocessing/discretizer.py:161: UserWarning: Some quantiles are equal. The number of bins will be smaller for sample [0]. Consider decreasing the number of bins or removing these samples.
  "of bins or removing these samples.".format(samples))
/home/asif/anaconda3/envs/pytorch/lib/python3.7/site-packages/pyts/preprocessing/discretizer.py:161: UserWarning: Some quantiles are equal. The number of bins will be smaller for sample [0]. Consider decreasing the number of bins or removing these samples.
  "of bins or removing these samples.".format(samples))
/home/asif/anaconda3/envs/pytorch/lib/python3.7/site-packages/pyts/preprocessing/discretizer.py:161: UserWarning: Some quantiles are equal. The number of bins will be smaller for sample [0]. Consider decreasing the number of bins or removing these samples.
  "of bins or removing these samples.".format(samples))
/home/asif/anaconda3/envs/pytorch/lib/python3.7/site-packages

/home/asif/anaconda3/envs/pytorch/lib/python3.7/site-packages/pyts/preprocessing/discretizer.py:161: UserWarning: Some quantiles are equal. The number of bins will be smaller for sample [0]. Consider decreasing the number of bins or removing these samples.
  "of bins or removing these samples.".format(samples))
/home/asif/anaconda3/envs/pytorch/lib/python3.7/site-packages/pyts/preprocessing/discretizer.py:161: UserWarning: Some quantiles are equal. The number of bins will be smaller for sample [0]. Consider decreasing the number of bins or removing these samples.
  "of bins or removing these samples.".format(samples))
/home/asif/anaconda3/envs/pytorch/lib/python3.7/site-packages/pyts/preprocessing/discretizer.py:161: UserWarning: Some quantiles are equal. The number of bins will be smaller for sample [0]. Consider decreasing the number of bins or removing these samples.
  "of bins or removing these samples.".format(samples))
/home/asif/anaconda3/envs/pytorch/lib/python3.7/site-packages

/home/asif/anaconda3/envs/pytorch/lib/python3.7/site-packages/pyts/preprocessing/discretizer.py:161: UserWarning: Some quantiles are equal. The number of bins will be smaller for sample [0]. Consider decreasing the number of bins or removing these samples.
  "of bins or removing these samples.".format(samples))
/home/asif/anaconda3/envs/pytorch/lib/python3.7/site-packages/pyts/preprocessing/discretizer.py:161: UserWarning: Some quantiles are equal. The number of bins will be smaller for sample [0]. Consider decreasing the number of bins or removing these samples.
  "of bins or removing these samples.".format(samples))
/home/asif/anaconda3/envs/pytorch/lib/python3.7/site-packages/pyts/preprocessing/discretizer.py:161: UserWarning: Some quantiles are equal. The number of bins will be smaller for sample [0]. Consider decreasing the number of bins or removing these samples.
  "of bins or removing these samples.".format(samples))
/home/asif/anaconda3/envs/pytorch/lib/python3.7/site-packages

/home/asif/anaconda3/envs/pytorch/lib/python3.7/site-packages/pyts/preprocessing/discretizer.py:161: UserWarning: Some quantiles are equal. The number of bins will be smaller for sample [0]. Consider decreasing the number of bins or removing these samples.
  "of bins or removing these samples.".format(samples))
/home/asif/anaconda3/envs/pytorch/lib/python3.7/site-packages/pyts/preprocessing/discretizer.py:161: UserWarning: Some quantiles are equal. The number of bins will be smaller for sample [0]. Consider decreasing the number of bins or removing these samples.
  "of bins or removing these samples.".format(samples))
/home/asif/anaconda3/envs/pytorch/lib/python3.7/site-packages/pyts/preprocessing/discretizer.py:161: UserWarning: Some quantiles are equal. The number of bins will be smaller for sample [0]. Consider decreasing the number of bins or removing these samples.
  "of bins or removing these samples.".format(samples))
/home/asif/anaconda3/envs/pytorch/lib/python3.7/site-packages

...

/home/asif/anaconda3/envs/pytorch/lib/python3.7/site-packages/pyts/preprocessing/discretizer.py:161: UserWarning: Some quantiles are equal. The number of bins will be smaller for sample [0]. Consider decreasing the number of bins or removing these samples.
  "of bins or removing these samples.".format(samples))
/home/asif/anaconda3/envs/pytorch/lib/python3.7/site-packages/pyts/preprocessing/discretizer.py:161: UserWarning: Some quantiles are equal. The number of bins will be smaller for sample [0]. Consider decreasing the number of bins or removing these samples.
  "of bins or removing these samples.".format(samples))
/home/asif/anaconda3/envs/pytorch/lib/python3.7/site-packages/pyts/preprocessing/discretizer.py:161: UserWarning: Some quantiles are equal. The number of bins will be smaller for sample [0]. Consider decreasing the number of bins or removing these samples.
  "of bins or removing these samples.".format(samples))
/home/asif/anaconda3/envs/pytorch/lib/python3.7/site-packages

/home/asif/anaconda3/envs/pytorch/lib/python3.7/site-packages/pyts/preprocessing/discretizer.py:161: UserWarning: Some quantiles are equal. The number of bins will be smaller for sample [0]. Consider decreasing the number of bins or removing these samples.
  "of bins or removing these samples.".format(samples))
/home/asif/anaconda3/envs/pytorch/lib/python3.7/site-packages/pyts/preprocessing/discretizer.py:161: UserWarning: Some quantiles are equal. The number of bins will be smaller for sample [0]. Consider decreasing the number of bins or removing these samples.
  "of bins or removing these samples.".format(samples))
/home/asif/anaconda3/envs/pytorch/lib/python3.7/site-packages/pyts/preprocessing/discretizer.py:161: UserWarning: Some quantiles are equal. The number of bins will be smaller for sample [0]. Consider decreasing the number of bins or removing these samples.
  "of bins or removing these samples.".format(samples))
/home/asif/anaconda3/envs/pytorch/lib/python3.7/site-packages

/home/asif/anaconda3/envs/pytorch/lib/python3.7/site-packages/pyts/preprocessing/discretizer.py:161: UserWarning: Some quantiles are equal. The number of bins will be smaller for sample [0]. Consider decreasing the number of bins or removing these samples.
  "of bins or removing these samples.".format(samples))
/home/asif/anaconda3/envs/pytorch/lib/python3.7/site-packages/pyts/preprocessing/discretizer.py:161: UserWarning: Some quantiles are equal. The number of bins will be smaller for sample [0]. Consider decreasing the number of bins or removing these samples.
  "of bins or removing these samples.".format(samples))
/home/asif/anaconda3/envs/pytorch/lib/python3.7/site-packages/pyts/preprocessing/discretizer.py:161: UserWarning: Some quantiles are equal. The number of bins will be smaller for sample [0]. Consider decreasing the number of bins or removing these samples.
  "of bins or removing these samples.".format(samples))
/home/asif/anaconda3/envs/pytorch/lib/python3.7/site-packages

/home/asif/anaconda3/envs/pytorch/lib/python3.7/site-packages/pyts/preprocessing/discretizer.py:161: UserWarning: Some quantiles are equal. The number of bins will be smaller for sample [0]. Consider decreasing the number of bins or removing these samples.
  "of bins or removing these samples.".format(samples))
/home/asif/anaconda3/envs/pytorch/lib/python3.7/site-packages/pyts/preprocessing/discretizer.py:161: UserWarning: Some quantiles are equal. The number of bins will be smaller for sample [0]. Consider decreasing the number of bins or removing these samples.
  "of bins or removing these samples.".format(samples))
/home/asif/anaconda3/envs/pytorch/lib/python3.7/site-packages/pyts/preprocessing/discretizer.py:161: UserWarning: Some quantiles are equal. The number of bins will be smaller for sample [0]. Consider decreasing the number of bins or removing these samples.
  "of bins or removing these samples.".format(samples))
/home/asif/anaconda3/envs/pytorch/lib/python3.7/site-packages

/home/asif/anaconda3/envs/pytorch/lib/python3.7/site-packages/pyts/preprocessing/discretizer.py:161: UserWarning: Some quantiles are equal. The number of bins will be smaller for sample [0]. Consider decreasing the number of bins or removing these samples.
  "of bins or removing these samples.".format(samples))
/home/asif/anaconda3/envs/pytorch/lib/python3.7/site-packages/pyts/preprocessing/discretizer.py:161: UserWarning: Some quantiles are equal. The number of bins will be smaller for sample [0]. Consider decreasing the number of bins or removing these samples.
  "of bins or removing these samples.".format(samples))
/home/asif/anaconda3/envs/pytorch/lib/python3.7/site-packages/pyts/preprocessing/discretizer.py:161: UserWarning: Some quantiles are equal. The number of bins will be smaller for sample [0]. Consider decreasing the number of bins or removing these samples.
  "of bins or removing these samples.".format(samples))
/home/asif/anaconda3/envs/pytorch/lib/python3.7/site-packages